<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/advanced_self_destruct_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
advanced_self_destruct_ai.py

A PyTorch module that self-destructs with:
  - Configurable decay schedules: linear, exponential, polynomial, sigmoid
  - Optional random-triggered collapse
  - Stateful resurrection support
  - Self-analysis on demise: returns a reflection tensor summarizing its lifecycle
"""

import torch
import torch.nn as nn
import random
import math
import argparse
import pprint


class AdvancedSelfDestructAI(nn.Module):
    def __init__(
        self,
        init_state: float = 1e6,
        phases: int = 5,
        decay: str = "linear",          # linear | exponential | polynomial | sigmoid
        poly_degree: float = 2.0,       # exponent if polynomial schedule
        sigmoid_steepness: float = 10.0,# controls slope of sigmoid
        random_death_prob: float = 0.0, # 0.0–1.0 chance per forward
        allow_resurrection: bool = False
    ):
        super().__init__()
        assert phases >= 1, "phases >= 1"
        assert decay in ("linear","exponential","polynomial","sigmoid")
        assert 0.0 <= random_death_prob <= 1.0

        # Core state
        self.register_buffer("null_state", torch.tensor(init_state))
        self.init_state = float(init_state)
        self.phases = phases
        self.decay = decay
        self.poly_degree = poly_degree
        self.sigmoid_k = sigmoid_steepness
        self.random_death_prob = random_death_prob

        # Lifecycle flags
        self.current_phase = 0
        self.alive = True
        self.allow_resurrection = allow_resurrection

        # Ephemeral logs
        self._logs = []

    def forward(self, x: torch.Tensor):
        if not self.alive:
            return None

        # Random-trigger collapse
        if random.random() < self.random_death_prob:
            self._snapshot("random collapse")
            reflection = self._reflect()
            self._self_destruct()
            return (x * 0.0), reflection

        # Normal expansion
        out = x * self.null_state
        self._snapshot(f"before phase {self.current_phase+1}")

        # Advance phase
        self.current_phase += 1
        with torch.no_grad():
            self._apply_decay()

        # Check natural end of life
        if self.current_phase >= self.phases or float(self.null_state) <= 0.0:
            self._snapshot("natural demise")
            reflection = self._reflect()
            self._self_destruct()
            return out, reflection

        return out

    def _apply_decay(self):
        if self.decay == "linear":
            delta = self.init_state / self.phases
            new_val = float(self.null_state) - delta
            self.null_state.fill_(max(new_val, 0.0))

        elif self.decay == "exponential":
            self.null_state.mul_(0.5)

        elif self.decay == "polynomial":
            frac = (self.current_phase / self.phases) ** self.poly_degree
            new_val = self.init_state * max(1 - frac, 0.0)
            self.null_state.fill_(new_val)

        else:  # sigmoid
            t = self.current_phase / self.phases
            # logistic: 1 / (1 + exp(k*(t-0.5)))
            scale = 1 / (1 + math.exp(self.sigmoid_k * (t - 0.5)))
            self.null_state.fill_(self.init_state * scale)

    def _snapshot(self, note: str):
        self._logs.append({
            "phase": self.current_phase,
            "null_state": float(self.null_state),
            "note": note
        })

    def _reflect(self) -> torch.Tensor:
        # Summarize lifecycle logs into a tensor:
        states = [entry["null_state"] for entry in self._logs]
        if not states:
            return torch.zeros(5)

        tensor = torch.tensor([
            states[0],                  # initial
            states[-1],                 # final
            float(torch.mean(torch.tensor(states))),
            float(torch.std(torch.tensor(states))),
            len(states)                 # number of snapshots
        ], dtype=torch.float32)
        return tensor

    def _self_destruct(self):
        # Wipe logs and state
        del self._logs[:]
        self.alive = False

    def resurrect(self):
        """
        Revive the model if allow_resurrection=True.
        Resets null_state, phase, logs, and alive flag.
        """
        if not self.allow_resurrection:
            raise RuntimeError("Resurrection disallowed")

        with torch.no_grad():
            self.null_state.fill_(self.init_state)
        self.current_phase = 0
        self.alive = True
        self._logs.clear()

    @property
    def logs(self):
        return list(self._logs)


def run_demo(
    init, phases, decay, poly, sigmoid_k, rand, allow_res
):
    model = AdvancedSelfDestructAI(
        init_state=init,
        phases=phases,
        decay=decay,
        poly_degree=poly,
        sigmoid_steepness=sigmoid_k,
        random_death_prob=rand,
        allow_resurrection=allow_res
    )
    sample = torch.tensor([init])

    print("\n=== Advanced Self-Destruct AI Demo ===")
    print(f"init={init}, phases={phases}, decay={decay}")
    if decay=="polynomial":
        print(f" poly_degree={poly}")
    if decay=="sigmoid":
        print(f" sigmoid_k={sigmoid_k}")
    print(f"rand_death={rand:.2%}, allow_res={allow_res}\n")

    step = 0
    while True:
        step += 1
        result = model(sample)
        # detect tuple (dying) vs single tensor
        if isinstance(result, tuple):
            out, reflection = result
            print(f"Step {step:3d}: final output = {out.item():.3e}")
            print(f"Reflection tensor = {reflection.tolist()}")
            break
        elif result is None:
            print(f"Step {step:3d}: dead with no reflection.")
            break

        print(f"Step {step:3d}: output = {result.item():.3e} | null_state = {model.null_state.item():.3e}")

        if step > phases + 5:
            print("Max steps reached, exiting.")
            break

    # Demonstrate resurrection if allowed
    if allow_res:
        print("\nResurrecting model...")
        model.resurrect()
        print(f"State after resurrection: null_state={model.null_state.item():.3e}, alive={model.alive}")
        print("Logs cleared:", model.logs)

    del model
    print("\nDemo complete. All traces removed.\n")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--init", type=float, default=1e6)
    parser.add_argument("--phases", type=int, default=5)
    parser.add_argument("--decay", choices=["linear","exponential","polynomial","sigmoid"], default="linear")
    parser.add_argument("--poly", type=float, default=2.0)
    parser.add_argument("--sigmoid_k", type=float, default=10.0)
    parser.add_argument("--rand", type=float, default=0.0)
    parser.add_argument("--allow_res", action="store_true")
    args, _ = parser.parse_known_args()

    run_demo(
        init=args.init,
        phases=args.phases,
        decay=args.decay,
        poly=args.poly,
        sigmoid_k=args.sigmoid_k,
        rand=args.rand,
        allow_res=args.allow_res
    )